In [1]:
pip install pyodbc

In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup

import pandas as pd

import pyodbc

import requests

import sqlalchemy
from sqlalchemy import create_engine

import time

In [5]:
# Read website into 'BeautifulSoup' to help parse the website
website_soup = BeautifulSoup(requests.get('https://vcnewsdaily.com/').text)

In [6]:
# Read funding activity and store into list
funding_activity = website_soup.find_all('div',class_='col-md-8')

In [7]:
# For loop to read in 'funding_activity', get information(date, headline, website link), and store into list
funding_activity_list = [[funding_activity[i].small.text,funding_activity[i].h5.text,funding_activity[i].a.get('href')] for i in range(len(funding_activity))]
df_funding_activity_list = pd.DataFrame(funding_activity_list,columns=['Funding Date','Funding Title','Website Article Link'])
df_funding_activity_list.head()

,Funding Date,Funding Title,Website Article Link
0,13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...
1,13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...
2,13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...
3,13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...
4,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...


In [8]:
df_funding_activity_list.shape

(30, 3)

In [9]:
# Articles from VC News Daily put into series
website_article_links = df_funding_activity_list['Website Article Link']

In [10]:
# Articles from our CSV download put into a series
website_article_links_total = pd.read_csv('VC_News_Daily.csv')['Website Article Link']

In [11]:
# Checking for nbumber of links that aren't already saved
website_article_links_length = len(website_article_links[~website_article_links_total.isin(website_article_links)])
website_article_links_length

8

In [12]:
df_funding_activity_list = df_funding_activity_list[0:website_article_links_length]
df_funding_activity_list.head()

,Funding Date,Funding Title,Website Article Link
0,13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...
1,13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...
2,13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...
3,13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...
4,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...


In [13]:
funding_activity_funding=[]
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_text = BeautifulSoup(requests.get(website_article_links[i]).text)
        funding_headline = company_text.find_all(id='summary')[0].text.strip('\n').strip('\t')
        if len(funding_headline.split(', ')) > 1:
            company_city = funding_headline.title().split(', ')[0]
            company_state = funding_headline.upper().split(', ')[1]
        else:
            company_city = 'N/A'
            company_state = 'N/A'
        website_company_link = company_text.find_all(id='fullArticle')[0].a.get('href')
        funding_article = company_text.find_all(id='fullArticle')[1].text
        funding_activity_funding.append([funding_headline,company_city,company_state,website_company_link,funding_article])

df_funding_activity_funding = pd.DataFrame(funding_activity_funding,columns=['Funding Headline','City','State','Website Company Link','Funding Article'])
df_funding_activity_funding.head()

,Funding Headline,City,State,Website Company Link,Funding Article
0,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
1,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
2,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
3,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
4,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."


In [14]:
df_funding_activity_funding.shape

(8, 5)

In [15]:
website_company_links = df_funding_activity_funding['Website Company Link']

In [16]:
funding_company_list = []
if website_article_links_length > 0:
    for i in range(len(df_funding_activity_list)):
        company_info = BeautifulSoup(requests.get(website_company_links[i]).text)
        company_name = company_info.find(class_='col-md-7 d-flex align-items-center').h5.text
        company_desc = company_info.find(class_='mt-5').p.text
        funding_type = company_info.find_all('td')[-2].text.strip('\n')
        funding_amount = company_info.find_all('td')[-3].text.strip('\n')
        funding_company_list.append([company_name,company_desc,funding_type,funding_amount])

df_funding_company_list = pd.DataFrame(funding_company_list,columns=['Company','Company Description','Latest Funding Round','Latest Funding Amount'])
df_funding_company_list.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount
0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000"
1,FalconX,FalconX is a digital asset trading platform th...,,"$17,000,000"
2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000"
3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000"
4,Transect,"Founded in San Antonio in 2016, Transect provi...",,undisclosed


In [17]:
df_funding_company_list.shape

(8, 4)

In [18]:
df_funding_data = pd.concat([df_funding_company_list,df_funding_activity_list,df_funding_activity_funding], axis=1)
df_funding_data.head()

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000",13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
1,FalconX,FalconX is a digital asset trading platform th...,,"$17,000,000",13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000",13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000",13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
4,Transect,"Founded in San Antonio in 2016, Transect provi...",,undisclosed,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."


In [19]:
df_funding_data.shape

(8, 12)

In [72]:
total_funding_data = pd.read_csv('VC_News_Daily.csv')

In [73]:
total_funding_data = pd.concat([df_funding_data,total_funding_data],axis=0).reset_index()

In [78]:
# Temporary fix for duplicates (when needed) # total_funding_data = total_funding_data.iloc[8:].reset_index(drop=True)

In [79]:
total_funding_data.head(10)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000",13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
1,FalconX,FalconX is a digital asset trading platform th...,NaN,"$17,000,000",13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000",13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000",13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
4,Transect,"Founded in San Antonio in 2016, Transect provi...",NaN,undisclosed,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."
5,Vennfi,Vennfi's flagship platform Charityvest offers ...,Pre-Seed,"$1,100,000",13 May 2020,Vennfi Closes Pre-Seed Round of $1.1M,https://vcnewsdaily.com/vennfi/venture-capital...,"ATLANTA, GA, Vennfi announced that it closed a...",Atlanta,GA,https://vcnewsdaily.com/Vennfi/venture-funding...,"Vennfi, the financial technology company for t..."
6,QurAlis,QurAlis Corporation is a Cambridge (USA) based...,Series A,"$42,000,000",13 May 2020,QurAlis Collects $42M in Series A,https://vcnewsdaily.com/quralis/venture-capita...,"CAMBRIDGE, MA, QurAlis Corporation announced t...",Cambridge,MA,https://vcnewsdaily.com/QurAlis/venture-fundin...,"QurAlis Corporation, a biotech company focused..."
7,Tock,"Tock is a complete reservation, guest, and tab...",NaN,"$10,000,000",13 May 2020,Tock Announces $10M in Funding,https://vcnewsdaily.com/tock/venture-capital-f...,"CHICAGO, IL, Tock announced an oversubscribed ...",Chicago,IL,https://vcnewsdaily.com/Tock/venture-funding.php,"Tock, the premier restaurant and winery bookin..."
8,SiMa.ai,SiMa.ai is a machine learning company enabling...,Series A,"$30,000,000",12 May 2020,SiMa.ai Raises $30M Series A,https://vcnewsdaily.com/sima.ai/venture-capita...,"SAN JOSE, CA, SiMa.ai, the machine learning co...",San Jose,CA,https://vcnewsdaily.com/SiMa.ai/venture-fundin...,The Series A funding round was led by Dell Tec...
9,Stellar Health,"Stellar Health is a point-of-care, cloud-based...",Series A,"$10,000,000",12 May 2020,Stellar Health Nabs $10M Series A Round,https://vcnewsdaily.com/stellar-health/venture...,"NEW YORK, NY, Stellar Health, a rapidly growin...",New York,NY,https://vcnewsdaily.com/Stellar%20Health/ventu...,"The round was led by Point72 Ventures, a VC fi..."


In [80]:
total_funding_data.shape

(78, 12)

In [55]:
total_funding_data.to_csv('VC_News_Daily.csv',index=False)

In [56]:
pd.read_csv('VC_News_Daily.csv').head(15)

,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000",13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
1,FalconX,FalconX is a digital asset trading platform th...,NaN,"$17,000,000",13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000",13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000",13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
4,Transect,"Founded in San Antonio in 2016, Transect provi...",NaN,undisclosed,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."
5,Vennfi,Vennfi's flagship platform Charityvest offers ...,Pre-Seed,"$1,100,000",13 May 2020,Vennfi Closes Pre-Seed Round of $1.1M,https://vcnewsdaily.com/vennfi/venture-capital...,"ATLANTA, GA, Vennfi announced that it closed a...",Atlanta,GA,https://vcnewsdaily.com/Vennfi/venture-funding...,"Vennfi, the financial technology company for t..."
6,QurAlis,QurAlis Corporation is a Cambridge (USA) based...,Series A,"$42,000,000",13 May 2020,QurAlis Collects $42M in Series A,https://vcnewsdaily.com/quralis/venture-capita...,"CAMBRIDGE, MA, QurAlis Corporation announced t...",Cambridge,MA,https://vcnewsdaily.com/QurAlis/venture-fundin...,"QurAlis Corporation, a biotech company focused..."
7,Tock,"Tock is a complete reservation, guest, and tab...",NaN,"$10,000,000",13 May 2020,Tock Announces $10M in Funding,https://vcnewsdaily.com/tock/venture-capital-f...,"CHICAGO, IL, Tock announced an oversubscribed ...",Chicago,IL,https://vcnewsdaily.com/Tock/venture-funding.php,"Tock, the premier restaurant and winery bookin..."
8,SiMa.ai,SiMa.ai is a machine learning company enabling...,Series A,"$30,000,000",12 May 2020,SiMa.ai Raises $30M Series A,https://vcnewsdaily.com/sima.ai/venture-capita...,"SAN JOSE, CA, SiMa.ai, the machine learning co...",San Jose,CA,https://vcnewsdaily.com/SiMa.ai/venture-fundin...,The Series A funding round was led by Dell Tec...
9,Stellar Health,"Stellar Health is a point-of-care, cloud-based...",Series A,"$10,000,000",12 May 2020,Stellar Health Nabs $10M Series A Round,https://vcnewsdaily.com/stellar-health/venture...,"NEW YORK, NY, Stellar Health, a rapidly growin...",New York,NY,https://vcnewsdaily.com/Stellar%20Health/ventu...,"The round was led by Point72 Ventures, a VC fi..."


In [57]:
# [Only once]
# Create excel file in order to upload into Microsoft SQL in order to upload dataset
total_funding_data.to_excel('VC_News_Daily.xlsx')

In [59]:
# Sensitive information withheld - via NULL
driver_name = '{NULL}'
server_name = 'NULL'
database_name = 'NULL'
trusted_connection = 'NULL'

In [60]:
# Connect to SQL Server
conn = pyodbc.connect(Driver=driver_name,
                      Server=server_name,
                      Database=database_name,
                      Trusted_Connection=trusted_connection)
cursor = conn.cursor()

In [84]:
# Before adding new information - SQL Table as is, without the reordering
sql_raw_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn)
sql_raw_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000",13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
1,41,Sleuth,"Deployment tracking, move fast without breakin...",Seed,"$3,000,000",07 May 2020,Sleuth Snares $3M Seed Financing,https://vcnewsdaily.com/sleuth/venture-capital...,"SAN FRANCISCO, CA, Sleuth announced it has rai...",San Francisco,CA,https://vcnewsdaily.com/Sleuth/venture-funding...,"According to TC, Sleuth announced it has raise..."
2,1,FalconX,FalconX is a digital asset trading platform th...,None,"$17,000,000",13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
3,2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000",13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
4,3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000",13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
5,4,Transect,"Founded in San Antonio in 2016, Transect provi...",None,undisclosed,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."
6,5,Vennfi,Vennfi's flagship platform Charityvest offers ...,Pre-Seed,"$1,100,000",13 May 2020,Vennfi Closes Pre-Seed Round of $1.1M,https://vcnewsdaily.com/vennfi/venture-capital...,"ATLANTA, GA, Vennfi announced that it closed a...",Atlanta,GA,https://vcnewsdaily.com/Vennfi/venture-funding...,"Vennfi, the financial technology company for t..."
7,16,Travecta Therapeutics,Travecta Therapeutics is a biopharmaceutical c...,Series A,"$15,000,000",12 May 2020,Travecta Therapeutics Scores $15M Series A,https://vcnewsdaily.com/travecta-therapeutics/...,"NEW YORK, NY, Travecta Therapeutics has secure...",New York,NY,https://vcnewsdaily.com/Travecta%20Therapeutic...,"Travecta Therapeutics, a biopharmaceutical com..."
8,6,QurAlis,QurAlis Corporation is a Cambridge (USA) based...,Series A,"$42,000,000",13 May 2020,QurAlis Collects $42M in Series A,https://vcnewsdaily.com/quralis/venture-capita...,"CAMBRIDGE, MA, QurAlis Corporation announced t...",Cambridge,MA,https://vcnewsdaily.com/QurAlis/venture-fundin...,"QurAlis Corporation, a biotech company focused..."
9,7,Tock,"Tock is a complete reservation, guest, and tab...",None,"$10,000,000",13 May 2020,Tock Announces $10M in Funding,https://vcnewsdaily.com/tock/venture-capital-f...,"CHICAGO, IL, Tock announced an oversubscribed ...",Chicago,IL,https://vcnewsdaily.com/Tock/venture-funding.php,"Tock, the premier restaurant and winery bookin..."


In [85]:
# Sensitive information withheld
engine = sqlalchemy.create_engine('NULL')

In [86]:
total_funding_data.to_sql('VC_News_Daily',con=engine,if_exists='replace')

In [89]:
sql_total_funding_data = pd.read_sql('SELECT * FROM VC_News_Daily',conn).sort_values('index')
sql_total_funding_data.head(15)

,index,Company,Company Description,Latest Funding Round,Latest Funding Amount,Funding Date,Funding Title,Website Article Link,Funding Headline,City,State,Website Company Link,Funding Article
0,0,Quizlet,Quizlet's study tools and applications are use...,Series C,"$30,000,000",13 May 2020,Quizlet Raises $30M Series C Funding,https://vcnewsdaily.com/quizlet/venture-capita...,"SAN FRANCISCO, CA, Quizlet announced a Series ...",San Francisco,CA,https://vcnewsdaily.com/Quizlet/venture-fundin...,"Quizlet, the global learning platform known fo..."
2,1,FalconX,FalconX is a digital asset trading platform th...,None,"$17,000,000",13 May 2020,FalconX Secures $17M Funding Round,https://vcnewsdaily.com/falconx/venture-capita...,"SAN FRANCISCO, CA, Digital asset trading platf...",San Francisco,CA,https://vcnewsdaily.com/FalconX/venture-fundin...,"FalconX, a digital asset trading platform that..."
3,2,UpKeep,UpKeep was created to alleviate all of the pai...,Series B,"$36,000,000",13 May 2020,UpKeep Raises $36M Series B,https://vcnewsdaily.com/upkeep/venture-capital...,"LOS ANGELES, CA, UpKeep announced it raised $3...",Los Angeles,CA,https://vcnewsdaily.com/UpKeep/venture-funding...,"UpKeep, the award-winning platform for mainten..."
4,3,ConverseNow,ConverseNow connects restaurants with their cu...,Seed,"$3,250,000",13 May 2020,ConverseNow Raises $3.25M Seed Funding,https://vcnewsdaily.com/conversenow/venture-ca...,"AUSTIN, TX, ConverseNow Technologies announced...",Austin,TX,https://vcnewsdaily.com/ConverseNow/venture-fu...,"ConverseNow Technologies, Inc., an Austin-base..."
5,4,Transect,"Founded in San Antonio in 2016, Transect provi...",None,undisclosed,13 May 2020,Transect Closes Funding Round,https://vcnewsdaily.com/transect/venture-capit...,"SAN ANTONIO, TX, Transect has successfully sec...",San Antonio,TX,https://vcnewsdaily.com/Transect/venture-fundi...,"Transect, a B2B SaaS company that partners wit..."
6,5,Vennfi,Vennfi's flagship platform Charityvest offers ...,Pre-Seed,"$1,100,000",13 May 2020,Vennfi Closes Pre-Seed Round of $1.1M,https://vcnewsdaily.com/vennfi/venture-capital...,"ATLANTA, GA, Vennfi announced that it closed a...",Atlanta,GA,https://vcnewsdaily.com/Vennfi/venture-funding...,"Vennfi, the financial technology company for t..."
8,6,QurAlis,QurAlis Corporation is a Cambridge (USA) based...,Series A,"$42,000,000",13 May 2020,QurAlis Collects $42M in Series A,https://vcnewsdaily.com/quralis/venture-capita...,"CAMBRIDGE, MA, QurAlis Corporation announced t...",Cambridge,MA,https://vcnewsdaily.com/QurAlis/venture-fundin...,"QurAlis Corporation, a biotech company focused..."
9,7,Tock,"Tock is a complete reservation, guest, and tab...",None,"$10,000,000",13 May 2020,Tock Announces $10M in Funding,https://vcnewsdaily.com/tock/venture-capital-f...,"CHICAGO, IL, Tock announced an oversubscribed ...",Chicago,IL,https://vcnewsdaily.com/Tock/venture-funding.php,"Tock, the premier restaurant and winery bookin..."
10,8,SiMa.ai,SiMa.ai is a machine learning company enabling...,Series A,"$30,000,000",12 May 2020,SiMa.ai Raises $30M Series A,https://vcnewsdaily.com/sima.ai/venture-capita...,"SAN JOSE, CA, SiMa.ai, the machine learning co...",San Jose,CA,https://vcnewsdaily.com/SiMa.ai/venture-fundin...,The Series A funding round was led by Dell Tec...
11,9,Stellar Health,"Stellar Health is a point-of-care, cloud-based...",Series A,"$10,000,000",12 May 2020,Stellar Health Nabs $10M Series A Round,https://vcnewsdaily.com/stellar-health/venture...,"NEW YORK, NY, Stellar Health, a rapidly growin...",New York,NY,https://vcnewsdaily.com/Stellar%20Health/ventu...,"The round was led by Point72 Ventures, a VC fi..."


In [90]:
sql_total_funding_data.shape

(78, 13)